# 1. 뉴스기사 정보 스크랩하기(기사내용 포함x)
- Naver
- 원하는 언론사 선택 가능
- 날짜 선택 가능(시작과 끝)
- Key_word 이용해서 원하는 키워드로 검색 가능(해당 단어가 완전히 포함된 기사)

### 검색하기

In [31]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import math
import time

# Input값
key_word = '베믈리디'
datestart = '20180101'
dateend = '20181231'
news = ['데일리팜', '의협신문', '데일리메디', '메디칼타임즈', '메디칼업저버', '메디게이트뉴스', 'e의료정보',
        '의학신문', '메디파나뉴스', '청년의사', '메디칼트리뷴', '보건뉴스', '약사공론', '약업신문', '라포르시안',
        '메디소비자뉴스', '메디컬월드뉴스', '메디컬투데이', '바이오스펙테이터', '정신의학신문', '코메디닷컴', '후생신보']
#news = ['KBS', 'MBC', 'SBS', 'JTBC', '연합뉴스', '조선일보', '중앙일보', '동아일보', '한국경제', '매일경제']

# chrome 드라이버 키고 naver 접속
driver = webdriver.Chrome('chromedriver.exe')
driver.get('http://naver.com')

driver.find_element_by_id('query').send_keys(key_word) # 검색어 입력
driver.find_element_by_id('search_btn').click() # 검색 클릭

driver.find_element_by_css_selector('div.lnb_menu').find_element_by_link_text('뉴스').click() # 뉴스 카테고리 클릭

# 검색옵션 클릭
driver.find_element_by_id('_search_option_btn').click()

# 기간 클릭
driver.find_element_by_link_text('기간').click()

# 기간 입력
driver.find_element_by_id('news_input_period_begin').click()
for i in datestart:
    driver.find_element_by_id('news_input_period_begin').send_keys(i)
    time.sleep(0.2)

driver.find_element_by_id('news_input_period_end').click()
for j in dateend:
    driver.find_element_by_id('news_input_period_end').send_keys(j)
    time.sleep(0.2)

# 적용 클릭
driver.find_element_by_css_selector('div.item_option.set_calendar').find_element_by_class_name('_btn_submit').click()

# 언론사 선택
driver.find_element_by_class_name('option_menu').find_element_by_link_text('언론사').click()

driver.find_element_by_id('order_cat')
for new in news:
    driver.find_element_by_xpath('//label[@title="{}"]'.format(new)).click()


driver.find_element_by_class_name('view_btn').find_element_by_css_selector('button.impact._submit_btn').click()

# 상세검색
#driver.find_element_by_id('snb').find_element_by_link_text('상세검색').click()

#driver.find_element_by_id('inpop1').send_keys(key_word)

#driver.find_element_by_css_selector('button.btn_ft.ty_green._search').click()

# 오래된 순으로 정렬
driver.find_element_by_link_text('오래된순').click()

### 내용 가져오기 - 댓글 제외
- 제목
- URL
- 날짜
- 언론사 이름

In [30]:
# 내용 가져오기
title_list = []
url_list = []
date_list = []
reporter_list = []

total_number_raw = driver.find_element_by_css_selector('div.title_desc.all_my').text
total_number = int(re.search(r'/ (.*?$)', total_number_raw).group(1)[:-1].replace(',', ''))
total_page = math.ceil(total_number/10)

for c in range(total_page):
    time.sleep(0.3)
    print('\r{}/{}---title({})/url({})/reporter({})/date({})'.format(c+1, total_page,len(title_list), len(url_list), len(reporter_list), len(date_list)), end = '')
    new10 = driver.find_element_by_css_selector('ul.type01').find_elements_by_tag_name('li')
    
    for n in new10:
        title_list.append(n.find_element_by_css_selector('a._sp_each_title').get_attribute('title'))
        try:
            link = n.find_element_by_link_text('네이버뉴스').get_attribute('href')
            url_list.append(link)
            
            
        except:
            link = n.find_element_by_css_selector('a._sp_each_url._sp_each_title').get_attribute('href')
            url_list.append(link)
            
        
        for strings in n.find_element_by_css_selector('dd.txt_inline').text.split():
            
            for rep in news:
                if re.match(rep, strings):
                    reporter_list.append(rep)
                
            if re.search(r'^[0-9]+\.[0-9]+\.[0-9]+\.$', strings):
                date_list.append(strings[:-1].replace('.', '-'))
                
    if len(title_list) != len(reporter_list):
        print(c)
    driver.find_element_by_link_text('다음페이지').click()

1/8---title(0)/url(0)/reporter(0)/date(0)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"다음페이지"}
  (Session info: chrome=78.0.3904.108)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.18362 x86_64)


### 내용 가져오기 - 댓글 포함

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 내용 가져오기
title_list = []
url_list = []
date_list = []
reporter_list = []
comment_list = []
while_error = 1

total_number_raw = driver.find_element_by_css_selector('div.title_desc.all_my').text
total_number = int(re.search(r'/ (.*?$)', total_number_raw).group(1)[:-1].replace(',', ''))
total_page = math.ceil(total_number/10)


current_window = driver.current_window_handle
for c in range(total_page):
    time.sleep(0.3)
    print('\r{}/{}---title({})/url({})/reporter({})/date({})/comment({}))'.format(c+1, total_page,len(title_list), len(url_list), len(reporter_list), len(date_list), len(comment_list)), end = '')
    new10 = driver.find_element_by_css_selector('ul.type01').find_elements_by_tag_name('li')
    
    for n in new10:
        
        title_list.append(n.find_element_by_css_selector('a._sp_each_title').get_attribute('title'))
        try:
            link = n.find_element_by_link_text('네이버뉴스').get_attribute('href')
            url_list.append(link)
            
            # 댓글 추가
            try:
                n.find_element_by_link_text('네이버뉴스').click()
                driver.switch_to.window(driver.window_handles[1])
                
                try:
                    element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "lo_txt"))
                    )
                except:
                    print('댓글창이 안 뜬다...')
                    
                while True:
                    if len(driver.find_element_by_css_selector('span.lo_txt').text) > 0:
                        while_error = 1
                        break
                    else:
                        print('\r아직 안 나왔어{}'.format(while_error), end = '')
                        while_error += 1
                        if while_error == 1000:
                            print('while 1000번 돌았어...')
                            while_error = 1
                            break                    
                    
                    try:
                        type(int(driver.find_element_by_css_selector('span.lo_txt').text)) == type(int(1))
                        break
                    except:
                        print('\r숫자가 안 나왔엉', end = '')
                        while_error += 1
                        if while_error == 1000:
                            print('while 1000번 돌았어...')
                            break
                            
                            
                            
                comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
                driver.close()
                driver.switch_to.window(current_window)
            except:
                print('댓글실패')
                driver.close()
                driver.switch_to.window(current_window)
                comment_list.append(99999)
            
        except:
            link = n.find_element_by_css_selector('a._sp_each_url._sp_each_title').get_attribute('href')
            url_list.append(link)
            
        
        for strings in n.find_element_by_css_selector('dd.txt_inline').text.split():
            
            for rep in news:
                if re.match(rep, strings):
                    reporter_list.append(rep)
                
            if re.search(r'^[0-9]+\.[0-9]+\.[0-9]+\.$', strings):
                date_list.append(strings[:-1].replace('.', '-'))
                
    if len(title_list) != len(reporter_list):
        print(c)
    try:
        driver.find_element_by_link_text('다음페이지').click()
    except:
        print('\r끝났습니다.{}/{}'.format(c, len(total_page)))

### CSV파일로 저장하기

In [28]:
data = {'date' : date_list, 'title' : title_list, 'repoter' : reporter_list, 'URL' : url_list}
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
date       71 non-null object
title      71 non-null object
repoter    71 non-null object
URL        71 non-null object
dtypes: object(4)
memory usage: 2.3+ KB


In [29]:
#df.loc[df['comment'] == '댓글', 'comment'] = 0
#df.loc[df['comment'] == 99999, 'comment'] = 0
file_name = '{}2018.csv'.format(key_word)
df.to_csv(file_name, encoding = 'utf-8', index = False)

### Comment와 content 파일 합치기

In [ ]:
df_comment = pd.read_csv('보장성강화_댓글추가_1164개.csv', encoding = 'utf-8', engine = 'python')
df_comment.head()

In [ ]:
df_content = pd.read_csv('보장성강화_content포함_댓글제외.csv', encoding = 'utf-8', engine = 'python', names = ['date', 'title', 'reporter', 'URL', 'content'])
df_content.head()

In [ ]:
result = pd.merge(df_comment, df_content, on = ['URL', 'date', 'title'], how = 'left')
result.drop('repoter', axis = 1, inplace = True)
result

In [ ]:
result.to_excel('보장성강화_Naver뉴스_댓글+내용(1164개).xlsx', encoding = 'utf-8')

### 중간에 삭제된 뉴스 추출

In [ ]:
df_1 = pd.read_csv('보장성강화_댓글제외.csv', engine = 'python', encoding = 'utf-8')
df_1.head()

In [ ]:
df_2 = pd.read_csv('보장성강화_댓글추가_1164개.csv', encoding = 'utf-8', engine = 'python')
df_2.head()

In [ ]:
a = df_1.URL.tolist()
b = df_2.URL.tolist()
c = list(set(a) - set(b))
c

In [ ]:
df_1[(df_1['URL'] == c[0]) | (df_1['URL'] == c[1])]

# 2. 기사내용 스크랩
- 위에서 클롤링한 URL주소를 이용하여 해당 뉴스내용 스크랩
- 네이버뉴스로 통하는 기사만 가능
- 안되는 언론사 많음

### URL를 이용해 스크랩

In [ ]:
content_list = []
content_url = []
comment_list = []
url_error_list = []

driver = webdriver.Chrome('chromedriver.exe')
for u in url_list:
    driver.get(u)
    try:
        content_list.append(driver.find_element_by_id('articleBodyContents').text.replace('\n', ' '))
        comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
        content_url.append(u)
        print('\r진행상황 {}/{}'.format(len(content_list), len(url_list)), end = '')
    except:
        url_error_list.append(u)
        print('에러발생{}'.format(len(url_error_list)))

### 에러URL 추가적으로 크롤링

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')
url_error_list2 = []
for u in url_error_list:
    driver.get(u)
    try:        
        content_list.append(driver.find_element_by_id('articleBodyContents').text.replace('\n', ' '))
        comment_list.append(driver.find_element_by_css_selector('span.lo_txt').text)
        content_url.append(u)
        print('\r진행상황 {}/{}'.format(len(content_list), len(url_list)), end = '')
    except:
        url_error_list2.append(u)
        print('또 에러발생{}'.format(len(url_error_list2)))

In [ ]:
driver.get('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=001&aid=0009455538')

In [ ]:
driver.find_element_by_css_selector('span.lo_txt').text

### CSV파일로 저장하기

In [ ]:
df_content = pd.DataFrame({'URL' : content_url, 'content' : content_list})
df_content.head()

# 3. 동영상 링크 스크랩 - 스토리룸
- 해당 사이트의 동영상 링크와 해당 제목 스크랩

### 사이트 접속

In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import math
import time

URL_link = 'http://medicare1.nhis.or.kr/hongbo/static/html/minisite/sub/bj02.html'
driver = webdriver.Chrome('chromedriver.exe')

driver.get(URL_link)
driver.find_element_by_css_selector('ul.content-desc.story-desc.clearfix')

### 카테고리 세분화

In [ ]:
c_list = []
for a in driver.find_elements_by_css_selector('ul.content-desc.story-desc.clearfix > li'):
    c_list.append(a.get_attribute('class').split()[1])
class_name_list = set(c_list)
class_name_list

### 자료수집

In [ ]:
content_list = driver.find_elements_by_css_selector('ul.content-desc.story-desc.clearfix > li' )

title_list = []
url_list = []
class_list1 = []
class_list2 = []

for content in content_list:
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    class_list1.append(content.get_attribute('class').split()[0])
    class_list2.append(content.get_attribute('class').split()[1])
print(len(url_list), len(title_list), len(class_list1))

### CSV파일로 저장

In [ ]:
data = {'title' : title_list, 'url' : url_list, 'class1' : class_list1, 'class2' : class_list2}
df = pd.DataFrame(data)
df.to_csv('스토리룸_유형+주제.csv', encoding = 'utf-8')

# 동영상 링크 스크램 - 뉴스룸
1. 카테고리별 형태가 달라서 각각 따로 처리

### 접속준비

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')

URL_link = 'http://medicare1.nhis.or.kr/hongbo/static/html/minisite/sub/bj03_{}.html'
title_list = []
url_list = []
class_list = []
cate = ['브리핑', '보도자료', '보도해명/설명자료', 'TV,라디오 등 방송인터뷰' ,'팩트체크']

### 자료수집 - 브리핑

In [ ]:
driver.get(URL_link.format(1))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[0])
len(url_list)

### 자료수집 - 보도자료

In [ ]:
driver.get(URL_link.format(2))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li.type-news.news-tab2 > ol > li > a')

for content in content_list:
    title_list.append(content.text[2:])
    url_list.append(content.get_attribute('href'))
    class_list.append(cate[1])
print(len(title_list), len(url_list), len(class_list))

### 자료수집 - 보도해명/설명자료

In [ ]:
driver.get(URL_link.format(3))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li.type-news.news-tab2 > ol > li > a')

for content in content_list:
    title_list.append(content.text[2:])
    url_list.append(content.get_attribute('href'))
    class_list.append(cate[2])
print(len(title_list), len(url_list), len(class_list))

### 자료수집 - TV,라디오 등 방송인터뷰

In [ ]:
driver.get(URL_link.format(4))
content_list = driver.find_elements_by_css_selector('ul.content-desc > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[3])
len(url_list)

### 자료수집 - 팩트체크

In [ ]:
driver.get(URL_link.format(5))
content_list = driver.find_elements_by_css_selector('ul.content-desc.story-desc.list3.clearfix > li')

for content in content_list:
    title_list.append(content.find_element_by_css_selector('div.story-title > a').text)
    url_list.append(content.find_element_by_css_selector('div.story-title > a').get_attribute('href'))
    class_list.append(cate[4])
len(url_list)

### CSV파일로 저장

In [ ]:
data = {'title' : title_list, 'url' : url_list, 'class' : class_list}
df = pd.DataFrame(data)
df.to_csv('뉴스룸.csv', encoding = 'utf-8')